<a href="https://colab.research.google.com/github/yonghoonlee-1/Cyberbullying-Detection/blob/main/Twitter_Sexism__CNN_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!rm -rf /content/gdrive
!ln -s "/gdrive/My Drive" /content/gdrive
%cd /content/gdrive

Mounted at /gdrive
/gdrive/My Drive


In [ ]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [ ]:
twitter=pd.read_csv("Cyberbullying_Twitter Sexism.csv", encoding='latin1')
twitter.head(10)

,sentence,sentiment
0,RT @colonelkickhead: Another bloody instant re...,none
1,@azzamalirhabi @JihadiA8 This video of the Pes...,none
2,Oh really? No more instant restaurants? THAT'S...,none
3,RT @Benfrancisallen: It hasn't been a good few...,none
4,RT @NoToFeminism: I donât need femisnsn beca...,none
5,@MariachiMacabre 19% is not the vast majority,none
6,RT @fruitondabottom: #FeminismisEQUALITYwhen M...,sexism
7,Gather round kids. It's story time brought to ...,none
8,Did you hear that? That's the sound of a bunch...,none
9,@jobbers7 And western feminists are silent.,none


In [ ]:
data = twitter[['sentence','sentiment']]

In [ ]:
import string

data['sentence'] = data['sentence'].apply(lambda x: x.lower())
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"\brt\b", "", x).strip()))
data['sentence'] = data['sentence'].apply((lambda x: re.sub(r"(https?|http)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]", "", x)))
data['sentence'] = data['sentence'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))
print(data[ data['sentiment'] == 'none'].size)
print(data[ data['sentiment'] == 'sexism'].size)

21504
6042


In [ ]:
def text_to_wordlist(text, remove_stopwords=True):    

    text = text.lower().split()
    
    if remove_stopwords:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    return(text)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
data['sentence'] = data.apply(lambda row: text_to_wordlist(row.sentence).split(), axis=1)

In [ ]:
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(data['sentence'].values)
X = tokenizer.texts_to_sequences(data['sentence'].values)
X = pad_sequences(X, maxlen=26)

In [ ]:
Y = pd.get_dummies(data['sentiment']).values

In [ ]:
print(X.shape)

(13773, 26)


In [ ]:
from sklearn.model_selection import KFold

skf = KFold(n_splits=5, shuffle=True)
accuracy = []
precision = []
recall = []
f1_score = []

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/gdrive/glove.twitter.27B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
from keras.layers import Bidirectional, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D 
from sklearn import metrics
from sklearn.utils import shuffle

X, Y = shuffle(X, Y, random_state=42)

embed_dim = 50
lstm_out = 50
for train, validation in skf.split(X, Y):
    model = Sequential()
    model.add(Embedding(len(tokenizer.word_index) + 1, embed_dim, weights=[embedding_matrix], input_length = X.shape[1]))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Bidirectional(LSTM(lstm_out, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model.fit(X[train], Y[train], epochs=10, batch_size=128)
    k_accuracy = '%.4f' % (model.evaluate(X[validation], Y[validation])[1])
    predict_test = (model.predict(X[validation]) >= 0.5) 
    k_precision = metrics.precision_score(Y[validation], predict_test, average=None)
    k_recall = metrics.recall_score(Y[validation], predict_test, average=None)
    k_f1_score = metrics.f1_score(Y[validation], predict_test, average=None)
    accuracy.append(k_accuracy)
    precision.append(k_precision)
    recall.append(k_recall)
    f1_score.append(k_f1_score)

Epoch 1/10
87/87 [==============================] - 10s 14ms/step - loss: 0.5258 - accuracy: 0.7738
Epoch 2/10
87/87 [==============================] - 1s 14ms/step - loss: 0.3772 - accuracy: 0.8449
Epoch 3/10
87/87 [==============================] - 1s 14ms/step - loss: 0.3163 - accuracy: 0.8719
Epoch 4/10
87/87 [==============================] - 1s 14ms/step - loss: 0.2406 - accuracy: 0.8993
Epoch 5/10
87/87 [==============================] - 1s 14ms/step - loss: 0.1867 - accuracy: 0.9271
Epoch 6/10
87/87 [==============================] - 1s 14ms/step - loss: 0.1325 - accuracy: 0.9501
Epoch 7/10
87/87 [==============================] - 1s 14ms/step - loss: 0.1073 - accuracy: 0.9607
Epoch 8/10
87/87 [==============================] - 1s 14ms/step - loss: 0.1222 - accuracy: 0.9530
Epoch 9/10
87/87 [==============================] - 1s 14ms/step - loss: 0.0901 - accuracy: 0.9640
Epoch 10/10
87/87 [==============================] - 1s 3ms/step - loss: 0.3265 - accuracy: 0.8973
Epoch 1/1

In [ ]:
def average(list):
    return (sum(list) / len(list))

In [ ]:
accuracy = list(map(float, accuracy))

In [ ]:
print(average(accuracy))

0.9028599999999999


In [ ]:
print(average(precision))

[0.93545006 0.79136688]


In [ ]:
print(average(recall))

[0.94101593 0.76797932]


In [ ]:
print(average(f1_score))

[0.93792323 0.77559184]
